In [7]:
import requests
import pandas as pd 
# Replace YOUR_API_KEY with your actual API key
api_key = 'B9367TAXK4ZMYQVI'
number_of_rows = 400
TESTING = True

In [2]:
colmn_rename = {
    'index': 'date',
    '1. open':'open',
    '2. high':'high',
    '3. low' : 'low',	
    '4. close' : 'close',
    '5. adjusted close' :	'adjusted_close',
    '6. volume'	: 'volume',
    '7. dividend amount'	: 'dividend_amount',
    '8. split coefficient' : 'split_coefficient'
}

In [3]:
symbol = 'IBM'

# Construct the API endpoint URL
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={api_key}'

# Make a GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the close data from the response JSON
    data = response.json()['Time Series (Daily)']
    close_data = {date: float(values['4. close']) for date, values in data.items()}
    print(f'data sanokes recieved from api {len(close_data)} for {symbol}')
    # print(close_data)
else:
    print(f'Request failed with status code {response.status_code} for {symbol}')


5892 <class 'dict'>


In [9]:
df_tmp = pd.DataFrame(data) \
    .T.reset_index() \
    .rename_axis('index', axis='columns') \
    .rename(columns = colmn_rename) 
#df_tmp.head()
cols = list(df_tmp.columns)
cols.remove('date')
df_tmp[cols] = df_tmp[cols].apply(pd.to_numeric, errors='coerce')
df_tmp['date'] = df_tmp['date'].apply(pd.to_datetime,format='%Y-%m-%d') 
df_tmp['close_percent_change'] = round(df_tmp.loc[::-1].close.pct_change() * 100, 4)
df_tmp.dtypes

index
date                    datetime64[ns]
open                           float64
high                           float64
low                            float64
close                          float64
adjusted_close                 float64
volume                           int64
dividend_amount                float64
split_coefficient              float64
close_percent_change           float64
dtype: object

In [10]:
df_fresh = df_tmp.iloc[:number_of_rows]
# df_fresh = df_tmp.iloc[:number_of_rows]
df2_old = df_tmp.tail(-number_of_rows)
df2_old

index,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,close_percent_change
400,2021-08-27,138.71,139.585,138.40,139.41,123.679976,2459643,0.0,1.0,0.4540
401,2021-08-26,139.97,140.800,138.71,138.78,123.121060,2498915,0.0,1.0,-0.7722
402,2021-08-25,139.92,140.800,139.46,139.86,124.079201,2012817,0.0,1.0,0.0143
403,2021-08-24,139.78,140.230,139.32,139.84,124.061458,2365638,0.0,1.0,0.1576
404,2021-08-23,139.62,140.150,138.80,139.62,123.866281,3039587,0.0,1.0,0.3666
...,...,...,...,...,...,...,...,...,...,...
5887,1999-11-05,92.75,92.940,90.19,90.25,49.122781,13737600,0.0,1.0,-1.4308
5888,1999-11-04,94.44,94.440,90.00,91.56,49.835810,16697600,0.0,1.0,-2.9776
5889,1999-11-03,95.87,95.940,93.50,94.37,51.365284,10369100,0.0,1.0,-0.4641
5890,1999-11-02,96.75,96.810,93.69,94.81,51.604774,11105400,0.0,1.0,-2.0052


In [11]:
df_tmp.to_parquet(f'data/all/parquet/{symbol}.parquet')
df2_old.to_parquet(f'data/old/parquet/{symbol}.parquet')
df_fresh.to_csv(f'data/new/csv/{symbol}.csv.gz')
if TESTING:
    df_fresh.to_csv(f'data/new/csv/{symbol}.csv')